In [1]:
from random import shuffle
import glob,h5py, cv2, scipy, pickle, itertools, re
import numpy as np
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm

filename = 'Datasets/'+'final_imit_spec2_128.pkl' # no resize
print('File to be created: ' + filename)

File to be created: Datasets/final_imit_spec2_128.pkl


In [2]:
# load images
subjects = ['10 dec emre mahbub', '11 jan mahbub emre ladi', '12 jan ozgur mahbub ladi emre', 
            '13 jan ridvan ademola sean akthar emre', '14 jan ashwanth zeki', '15 apr ozgur',
           '15 jan emin', '19 apr sean', '19 apr sevgi', '21 apr emin', '22 apr ladi', '22 apr sean',
           '23 apr alperen', '23 apr mahbub', '5 apr sevgi']
# subjects = ['6 apr blake', '15 apr darrin', '16 apr devon asl2', '20 apr caroline', '21 apr blake']
mainpath = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/'
im_files = []
for i in range(len(subjects)):
    addr = glob.glob(mainpath+subjects[i]+'/microDoppler/'+'*.png')
    im_files.append(addr)
im_files = sum(im_files, [])
print('Num. of label files: '+str(len(im_files)))
for i in range(0,5):    
    print(str(i+1)+'. '+im_files[i])

Num. of label files: 1320
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/microDoppler/11020014_1607634489_4.png
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/microDoppler/11020014_1607634489_5.png
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/microDoppler/11050014_1607635095_2.png
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/microDoppler/11020014_1607634489_1.png
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/10 dec emre mahbub/microDoppler/11050014_1607635095_5.png


In [3]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return np.array(dataframe.values)

In [4]:
labels = []
common = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/common labels/microDoppler/'
cnt = []
for i in tqdm(range(len(im_files))):
    seq = im_files[i][-22]
    it = im_files[i][-5]
    
    if seq == '1' and it == '1':
        labels.append(np.squeeze(load_file(common+'1_1.txt')))
        cnt.append(0)
    elif seq == '1' and it != '1':
        labels.append(np.squeeze(load_file(common+'1_2to5.txt')))
        cnt.append(1)
    elif seq == '2' and it == '1':
        labels.append(np.squeeze(load_file(common+'2_1.txt')))
        cnt.append(2)
    elif seq == '2' and it != '1':
        labels.append(np.squeeze(load_file(common+'2_2to5.txt')))
        cnt.append(3)
    elif seq == '3' and it == '1':
        labels.append(np.squeeze(load_file(common+'3_1.txt')))
        cnt.append(4)
    elif seq == '3' and it != '1':
        labels.append(np.squeeze(load_file(common+'3_2to5.txt')))
        cnt.append(5)
    elif seq == '4' and it == '1':
        labels.append(np.squeeze(load_file(common+'4_1.txt')))
        cnt.append(6)
    elif seq == '4' and it != '1':
        labels.append(np.squeeze(load_file(common+'4_2to5.txt')))
        cnt.append(7)
    elif seq == '5' and it == '1':
        labels.append(np.squeeze(load_file(common+'5_1.txt')))
        cnt.append(8)
    elif seq == '5' and it != '1':
        labels.append(np.squeeze(load_file(common+'5_2to5.txt')))
        cnt.append(9)
labels = np.array(labels)
print(labels.shape)
print(Counter(cnt))

100%|██████████| 1320/1320 [02:22<00:00,  9.25it/s]

(1320, 1482)
Counter({3: 212, 9: 212, 7: 212, 1: 212, 5: 208, 2: 53, 6: 53, 8: 53, 0: 53, 4: 52})


In [5]:
envelopes = []
envs = glob.glob('/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/*/envelopes/*.txt')
print(len(envs))
for i in tqdm(range(len(im_files)), position = 0, leave = True):
#     clear_output(wait=True)
    fname = im_files[i][-25:].replace('png','txt')
    idx = [x for x in range(len(envs)) if re.search(fname, envs[x])]
    assert len(idx) == 1
    envelopes.append(np.squeeze(load_file(envs[idx[0]])))
envelopes = np.array(envelopes) 

  0%|          | 1/1320 [00:00<02:50,  7.71it/s]

1970


100%|██████████| 1320/1320 [02:18<00:00,  9.52it/s]


In [6]:
train_spec, test_spec, y_train, y_test, env_train, env_test = train_test_split(im_files, labels, envelopes, test_size=0.2, random_state=1)

In [7]:
dur = 24 # sec
win = 0.2 # win len
n_step = int(dur/win)
des_w = 128*n_step
des_h = 128
print('num steps: '+str(n_step))
print('desired width: '+str(des_w))

num steps: 120
desired width: 15360


In [8]:
def resizer(im, des_w, des_h, n_step):
    resized = np.reshape(cv2.resize(im, (des_w, des_h), interpolation = cv2.INTER_AREA),(n_step, des_h, 128, 3))    
    return resized

In [9]:
cnt = 1
train_images = []

for image in train_spec:
    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    img = resizer(img, des_w, des_h, n_step)
    clear_output(wait=True)
    print('Loading training image '+str(cnt)+'/'+str(len(train_spec))+ ' '+image)
    print('orig shape: '+str(img.shape))
    train_images.append(img/255.)  
    cnt += 1
train_images = np.array(train_images)
cnt = 1
test_images = []

for image in test_spec:
    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    img = resizer(img, des_w, des_h, n_step)
    clear_output(wait=True)
    print('Loading test image '+str(cnt)+'/'+str(len(test_spec))+ ' '+image)
    print('orig shape: '+str(img.shape))
    test_images.append(img/255.)  
    cnt += 1
test_images = np.array(test_images)

print(train_images.shape)
print(test_images.shape)

Loading test image 264/264 /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/13 jan ridvan ademola sean akthar emre/microDoppler/11020003_1610581049_4.png
orig shape: (120, 128, 128, 3)
(1056, 120, 128, 128, 3)
(264, 120, 128, 128, 3)


In [10]:
data = [train_images[0:800], test_images[0:200], y_train[0:800], y_test[0:200], env_train[0:800], env_test[0:200]]
with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)
print(filename+' created.')

Datasets/final_imit_spec2_128.pkl created.


In [11]:
with open(filename, 'rb') as input:
    x = pickle.load(input)

In [12]:
x[0][0].shape

(120, 128, 128, 3)